# Calculator Lexer - Parser

## 

| Sintactic rules | Semantics rules |
| :-: | :-: |
| input -> input expr ; | Escribe(expr.s) |
| input -> $\epsilon$ | |
| expr -> $expr_1$ `+` sum | $expr.s = expr_1.s + sum.s$ |
| expr -> $expr_1$ `-` sum | $expr.s = expr_1.s - sum.s$ |
| expr -> sum |  expr.s = sum.s|
| sum -> $sum_1 `*` fact$ | $sum.s = sum_1.s * fact.s$ |
| sum -> $sum_1 `/` fact$ | $sum.s = sum_1.s / fact.s$ |
| sum -> fact | sum.s = fact.s|
| fact -> $- fact_1$ | fact.s = $- fact_1.s $|
| fact -> NUM | fact.s = NUM.lexval |
| fact -> `(` expr `)` | fact.s = expr.s |

In [2]:
from sly import Lexer, Parser